# Setup

In [1]:
using DifferentialEquations, DiffEqBayes, Plots, Turing, Interpolations

# Three node network

Based on lorenz example. Writing out the system of equations individually instead of looping through each node.

In [2]:
phi(a::Number) = (exp(2*a)-1)/(exp(2*a)+1)

phi (generic function with 1 method)

In [86]:
function additive_noise(du,u,p,t)
    s,g,W,I = p
    for i = 1:length(u)
        du[i] = 1
    end
end

function network_model(du,u,p,t)
    s,g,W,I = p
    for i=1:length(u)
        du[i] = -u[i] + s*phi(u[i]) + g*(sum(W*u)) + I(t)
    end
end

network_model (generic function with 1 method)

In [87]:
num_nodes = 3
u0 = repeat([0.0], num_nodes)
Tmax = 100.0
tspan = (0,Tmax)

s = 0.3
g = 0.7
W = [0.0 0.2 0.0; 0.4 0.0 0.0; 0.0 0.3 0.0]

ts = 1:1:Tmax
task = repeat([0.0], 100)
task[4] = 1
I = LinearInterpolation(ts, task, extrapolation_bc = 0)
dt = 0.5

p = [s,g,W,I]

prob_sde = SDEProblem(network_model,additive_noise,u0,tspan,p)


SDEProblem with uType Array{Float64,1} and tType Float64. In-place: true
timespan: (0.0, 100.0)
u0: [0.0, 0.0, 0.0]

In [88]:
solve(prob_sde)

LoadError: MethodError: no method matching +(::Float64, ::Array{Float64,1})
For element-wise addition, use broadcasting with dot syntax: scalar .+ array
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  +(::Float64, !Matched::Float64) at float.jl:401
  +(!Matched::ChainRulesCore.DoesNotExist, ::Any) at /Users/zeynepenkavi/.julia/packages/ChainRulesCore/eBwSt/src/differential_arithmetic.jl:23
  ...